In [ ]:
import numpy as np
from BPnumba.NumAG import createAG
from BPmodule.BPGen import PROBLEM2, PROBLEM3, PROBLEM4, PROBLEM5
from BPnumba.GeneticOperators import CreatePoblation, InstancePob,CreateHeuristicPob,Ind
from BPnumba.DFFA import createDFFA
from BPnumba.ABCnum import createDABC
from numba import njit, objmode, prange
import time
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt


In [18]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]
numTest = 50
n = 4
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = x + ".csv",
    X= y,
    delimiter=", ",
    header=" Fi BoxesPacked MaxItr Time(s) | 50 individuos",
    fmt='%g'
)

ag2 = zerovec(n, 2*numTest)
ab2 = zerovec(n, 2*numTest)
fa2 = zerovec(n, 2*numTest)
ag3 = zerovec(n,2*numTest)
ab3 = zerovec(n, 2*numTest)
fa3 = zerovec(n, 2*numTest)
ag4 = zerovec(n, 2*numTest)
ab4 = zerovec(n, 2*numTest)
fa4 = zerovec(n, 2*numTest)
ag5 = zerovec(n, 2*numTest)
ab5 = zerovec(n, 2*numTest)
fa5 = zerovec(n, 2*numTest)

In [9]:
@njit(parallel=True)
def Test(num_test: int, ag, sameInstance: List[int], maxItr: int, agL: List[List[float]], datos: List[List[List[int]]], contenedor: List[int],heuristic:int=0):
    n = len(datos)
    for _ in prange(num_test):
        Pob = InstancePob(sameInstance, datos, contenedor, heuristic)
        with objmode(time1='f8'):
            time1 = time.perf_counter()
        bestInd:Ind = ag.Train(maxItr, Pob, datos, contenedor)
        epochs = len(ag.bestfi)
        with objmode(last='f8'):
            last = time.perf_counter() - time1
        if heuristic == 0:
            agL[_] = np.array([bestInd.fi, n-bestInd.load,
                            epochs, last], dtype=np.float64)
        else:
             agL[num_test+_] = np.array([bestInd.fi, n-bestInd.load,
                            epochs, last], dtype=np.float64)


In [19]:
pr_selec = 0.85
pr_cross = 0.75
pr_mut = 0.2
gamma = 1/(n*n)
pop = 50
maxItr=1000

In [20]:
contenedor = P3A1[0][0]
datos = P3A1[1]
n = len(datos)
sameInstance = CreateHeuristicPob(pop, datos,True)

In [21]:
ag = createAG(pr_selec, pr_cross, pr_mut)
Test(numTest, ag,sameInstance, maxItr, ag3, datos, contenedor)
createText("AGp3",ag3)

ag = createAG(pr_selec, pr_cross, pr_mut,2)
Test(numTest, ag,sameInstance, maxItr, ag3, datos, contenedor,2)
createText("AGp2",ag3)

In [22]:
ffa = createDFFA(gamma)
Test(numTest, ffa,sameInstance, maxItr, fa3, datos, contenedor)
createText("DFFAp2",fa3)

ffa = createDFFA(gamma,2)
Test(numTest, ffa,sameInstance, maxItr, fa3, datos, contenedor,2)
createText("DFFAp2",fa3)

In [23]:
abc = createDABC(pop,n)
Test(numTest, abc,sameInstance, maxItr, ab2, datos, contenedor)
createText("DABCp2",ab2)

abc = createDABC(pop,n,2)
Test(numTest, abc,sameInstance, maxItr, ab2, datos, contenedor,2)
createText("DABCp2",ab2)